In [2]:
import cv2
import numpy as np

# Load the image
path='kotmale.jpg'
image=cv2.imread('kotmale.jpg')

# Original size of the image
height, width, channels = image.shape
print(f"Width: {width}, Height: {height}")

# Check if the image was loaded correctly
if image is None:
    print ("Error:please check the path.")
    exit()

Width: 1280, Height: 1280


In [3]:
# Resize the image
new_width=512
new_height=512
resized_image=cv2.resize(image,(new_width,new_height))

In [4]:
# Crop the image
def crop_circle (image, radius_factor=0.6):
    height,width=image.shape[:2]
    mask=np.zeros((height,width), dtype=np.uint8)
    center=(int(width/2), int(height/2))
    original_radius= min(center[0], center[1])
    radius=int(original_radius*radius_factor)
    cv2.circle(mask,center,radius,(255,),-1, cv2.LINE_AA)
    masked_image=cv2.bitwise_and(image,image,mask=mask)
    return masked_image

cropped_image=crop_circle(resized_image, radius_factor=0.6)
cv2.imwrite('cropped.jpg', cropped_image)

True

In [5]:
# Convert the cropped image to grayscale
gray_scaled_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)

In [6]:
# Apply histogram equalization
equalized_image = cv2.equalizeHist(gray_scaled_image)

In [7]:
blurred_image = cv2.GaussianBlur(equalized_image, (3, 3), 0)

In [8]:
# Perform edge detection on the blurred image
# Convert the normalized image back to an 8-bit image
blurred_image_8bit = (blurred_image * 255).astype(np.uint8)
edges = cv2.Canny(blurred_image_8bit, 50, 100)

In [9]:
# Adaptive Thresholding
adaptive_thresh_image = cv2.adaptiveThreshold(blurred_image_8bit, 255,
                                              cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                              cv2.THRESH_BINARY_INV, 11, 2)

In [10]:
# Apply morphological operations to clean up the binary image
kernel = np.ones((3, 3), np.uint8)
cleaned_image = cv2.morphologyEx(adaptive_thresh_image, cv2.MORPH_CLOSE, kernel, iterations=2)

In [11]:
# Calculate the percentage of area covered by fungi colonies
# Count the number of white pixels in the cleaned binary image
fungi_area = np.sum(cleaned_image == 255)
# Calculate the total number of pixels in the image
total_area = cleaned_image.size
# Calculate the percentage
fungi_percentage = (fungi_area / total_area) * 100
print(f"Percentage of area covered by fungi colonies: {fungi_percentage:.2f}%")

Percentage of area covered by fungi colonies: 28.20%


In [ ]:
# Display the images
cv2.imshow('Gray scale Image', gray_scaled_image)
cv2.imshow('Equalized Image', equalized_image)
cv2.imshow('Blurred Image', blurred_image)
cv2.imshow('Edges', edges)
cv2.imshow('Adaptive Threshold Image', adaptive_thresh_image)
cv2.imshow('Cleaned Image', cleaned_image)


cv2.waitKey(0)
cv2.destroyAllWindows()